# Chem Space (No Scaffold)
### By: Francisco Feitosa

### Import Libraries

In [1]:
import os
import time
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmilesFromSmiles
from rdkit import Chem, DataStructs
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem import rdDepictor
from rdkit.Chem import MACCSkeys
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.PandasTools import ChangeMoleculeRendering

import mols2grid
from tqdm.auto import tqdm
from ipywidgets import widgets
from typing import List

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
#import umap as umap
#import hdbscan

import matplotlib.pyplot as plt
import seaborn as sns
IPythonConsole.ipython_useSVG=True 
from IPython.display import SVG

#Bokeh library for plotting
import json
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.transform import factor_cmap
from bokeh.transform import transform
from bokeh.transform import LinearColorMapper
from bokeh.models import ColorBar
from bokeh.palettes import PiYG
from bokeh.plotting import figure, output_file, save
output_notebook()

Loading BokehJS ...

In [2]:
tqdm.pandas()

### Setup

In [3]:
fname = r"C:\Users\franc\OneDrive\Documentos\LabMol\IC-Skin\DADOS\GHS\curated_binary_GHS.csv"

In [4]:
df = pd.read_csv(fname)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471 entries, 0 to 470
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   SMILES   471 non-null    object
 1   Outcome  471 non-null    object
 2   Source   471 non-null    object
 3   CAS No   471 non-null    object
dtypes: object(4)
memory usage: 14.8+ KB


In [5]:
df=df.rename(columns={'Outcome':'ACTIVITY'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471 entries, 0 to 470
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   SMILES    471 non-null    object
 1   ACTIVITY  471 non-null    object
 2   Source    471 non-null    object
 3   CAS No    471 non-null    object
dtypes: object(4)
memory usage: 14.8+ KB


In [6]:
df1=df[['SMILES','ACTIVITY']]
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471 entries, 0 to 470
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   SMILES    471 non-null    object
 1   ACTIVITY  471 non-null    object
dtypes: object(2)
memory usage: 7.5+ KB


In [7]:
df1.isnull().any()

SMILES      False
ACTIVITY    False
dtype: bool

In [8]:
df1=df1.dropna(subset=['ACTIVITY'])
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471 entries, 0 to 470
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   SMILES    471 non-null    object
 1   ACTIVITY  471 non-null    object
dtypes: object(2)
memory usage: 7.5+ KB


In [9]:
df1['mol'] = df.SMILES.progress_apply(Chem.MolFromSmiles)

  0%|          | 0/471 [00:00<?, ?it/s]

In [10]:
df1.isnull().any()
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471 entries, 0 to 470
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   SMILES    471 non-null    object
 1   ACTIVITY  471 non-null    object
 2   mol       471 non-null    object
dtypes: object(3)
memory usage: 11.2+ KB


### Fingerprints

In [11]:
fp_type = "ECFP"
radii = 2
bits = 1024

In [12]:
for type in fp_type:
    if fp_type == "ECFP":
        fps = np.array([AllChem.GetMorganFingerprintAsBitVect(x, radius = radii, nBits = bits, useFeatures=False) for x in df1['mol']])
    if fp_type == "FCFP":
        fps = np.array([AllChem.GetMorganFingerprintAsBitVect(x, radius = radii, nBits = bits, useFeatures=True) for x in df1['mol']])
    if fp_type == "MACCS":
        fps = np.array([MACCSkeys.GenMACCSKeys(x) for x in df1['mol']])
    

### Calculate PCA

In [13]:
%%time
pca = PCA(n_components=2)
X_pca = pca.fit_transform(fps)
pca_df = pd.DataFrame(X_pca, columns= ['X_PCA','Y_PCA'])
pca_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471 entries, 0 to 470
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X_PCA   471 non-null    float64
 1   Y_PCA   471 non-null    float64
dtypes: float64(2)
memory usage: 7.5 KB
CPU times: total: 0 ns
Wall time: 372 ms


### Calculate TSNE

In [14]:
%%time
tsne = TSNE(random_state=0).fit_transform(fps)
tsne_df = pd.DataFrame(tsne, columns= ['X_TSNE', 'Y_TSNE'])

tsne_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471 entries, 0 to 470
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X_TSNE  471 non-null    float32
 1   Y_TSNE  471 non-null    float32
dtypes: float32(2)
memory usage: 3.8 KB
CPU times: total: 8.98 s
Wall time: 881 ms


### Calculate UMAP

In [15]:
"""%%time
umap_model = umap.UMAP(metric = "euclidean",
                      n_neighbors = 25,
                      n_components = 2,
                      low_memory = False,
                      min_dist = 0.9)
X_umap = umap_model.fit_transform(fps)
umap_df = pd.DataFrame(X_umap, columns= ['X_UMAP', 'Y_UMAP'])"""

'%%time\numap_model = umap.UMAP(metric = "euclidean",\n                      n_neighbors = 25,\n                      n_components = 2,\n                      low_memory = False,\n                      min_dist = 0.9)\nX_umap = umap_model.fit_transform(fps)\numap_df = pd.DataFrame(X_umap, columns= [\'X_UMAP\', \'Y_UMAP\'])'

### Prepare molecules to print

In [16]:
def _prepareMol(mol,kekulize):
    mc = Chem.Mol(mol.ToBinary())
    if kekulize:
        try:
            Chem.Kekulize(mc)
        except:
            mc = Chem.Mol(mol.ToBinary())
    if not mc.GetNumConformers():
        rdDepictor.Compute2DCoords(mc)
    return mc

def moltosvg(mol,molSize=(450,200),kekulize=True,drawer=None,**kwargs):
    mc = _prepareMol(mol,kekulize)
    if drawer is None:
        drawer = rdMolDraw2D.MolDraw2DSVG(molSize[0],molSize[1])
    drawer.DrawMolecule(mc,**kwargs)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText()
    return SVG(svg.replace('svg:',''))

In [17]:
svgs = [moltosvg(m).data for m in df1.mol]

In [18]:
df1=df1.reset_index()
df1.head()

,index,SMILES,ACTIVITY,mol
0,0,OC1=CC=CC(O)=C1O,1A,<rdkit.Chem.rdchem.Mol object at 0x000002AB226...
1,1,O=CC=C(C1=CC=CC=C1)C1=CC=CC=C1,1A,<rdkit.Chem.rdchem.Mol object at 0x000002AB226...
2,2,O=C1C=CC(=O)C=C1,1A,<rdkit.Chem.rdchem.Mol object at 0x000002AB226...
3,3,O=C(OOC(=O)C1=CC=CC=C1)C1=CC=CC=C1,1A,<rdkit.Chem.rdchem.Mol object at 0x000002AB226...
4,4,NC1=CC=CC=C1NC1=CC=CC=C1,1A,<rdkit.Chem.rdchem.Mol object at 0x000002AB226...


In [19]:
df_merge = pd.concat([df1, pca_df, tsne_df], axis=1)
df_merge.head()

,index,SMILES,ACTIVITY,mol,X_PCA,Y_PCA,X_TSNE,Y_TSNE
0,0,OC1=CC=CC(O)=C1O,1A,<rdkit.Chem.rdchem.Mol object at 0x000002AB226...,-1.253673,-0.153482,-6.269214,-13.437871
1,1,O=CC=C(C1=CC=CC=C1)C1=CC=CC=C1,1A,<rdkit.Chem.rdchem.Mol object at 0x000002AB226...,-1.356394,0.445698,-0.437766,-16.889528
2,2,O=C1C=CC(=O)C=C1,1A,<rdkit.Chem.rdchem.Mol object at 0x000002AB226...,-0.676710,-0.837569,-2.223163,-5.417204
3,3,O=C(OOC(=O)C1=CC=CC=C1)C1=CC=CC=C1,1A,<rdkit.Chem.rdchem.Mol object at 0x000002AB226...,-1.229565,1.263253,5.014414,-15.239364
4,4,NC1=CC=CC=C1NC1=CC=CC=C1,1A,<rdkit.Chem.rdchem.Mol object at 0x000002AB226...,-1.471168,0.080976,-5.372641,-18.337759


### Plot Map

In [20]:
color_mapper=LinearColorMapper(palette=PiYG[9],
                               low=df1.ACTIVITY.max(), high=df1.ACTIVITY.min())

ValueError: failed to validate LinearColorMapper(id='p1001', ...).low: expected either None or a value of type Float, got '1B'

In [ ]:
df1['s'] = 1
df1

In [ ]:
def plot_int_map(metodology):
    source = ColumnDataSource(data=dict(x=metodology[:,0], y=metodology[:,1], freq = df1.s, desc= df1.ACTIVITY,
                                    svgs=svgs, s=df1.s*6, c=df1.ACTIVITY))
    ChangeMoleculeRendering(renderer='PNG')



    hover = HoverTool(tooltips="""
        <div>
            <div>@svgs{safe}
            </div>
            <div>
                <span style="font-size: 17px; font-weight: bold;">Frequency @freq</span>
            </div>
            <div>
                <span style="font-size: 17px; font-weight: bold;">Activity @desc</span>
            </div>
        </div>
        </body>
        """)

    interactive_map = figure(width = 1000, height=1000, tools=['reset,box_zoom,wheel_zoom,zoom_in,zoom_out,pan',hover],
           title="Chemical Space " + str(fp_type) + " Radius: " + str(radii) + " bits: " + str(bits))



    interactive_map.circle('x', 'y', 
                           source=source,
                           size='s',
                           color=transform('c', color_mapper),
                           fill_alpha=0.5);

    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0), title ='Activity')
    interactive_map.add_layout(color_bar,'left')
    show(interactive_map)
    return interactive_map

In [ ]:
pca_map=plot_int_map(X_pca)

In [ ]:
tsne_map=plot_int_map(tsne)

In [ ]:
#umap_map= plot_int_map(X_umap)

In [ ]:
#Save Hmtl for later viewing
#output_file("./umap " + str(fp_type) + " " + str(radii) + " " + str(bits) + ".html")
#save(umap_map)

output_file("./tsne " + str(fp_type) + " " + str(radii) + " " + str(bits) + ".html")
save(tsne_map)

output_file("./pca " + str(fp_type) + " " + str(radii) + " " + str(bits) + ".html")
save(pca_map)
